In [42]:
# All American School of English — Email SDR core (Resend)
# --------------------------------------------------------
# What this file does:
# 1) Loads .env and sets company context/constants.
# 2) Defines fast function tools (no extra LLM calls):
#    - subject_writer(message)  -> subject line
#    - html_converter(body)     -> branded, responsive HTML
# 3) Provides Resend senders:
#    - send_html_email(subject, html_body): sends to RESEND_TO
#    - send_html_email_dynamic(to_email, subject, html_body, in_reply_to, references): threaded replies
# 4) Adds plain-text fallback for deliverability and sets reply_to for inbound routing.
#
# Env vars:
#   RESEND_API_KEY  (required)
#   RESEND_FROM     (verified sender; use onboarding@resend.dev for tests)
#   RESEND_TO       (default recipient for initial sends)
#   REPLY_TO        (inbox for human replies; defaults to COMPANY_EMAIL)
#   PRICE_PLN / PRICE_USD (optional display values)
#
# Notes:
# - Use your Email Manager / Turbo pipeline to generate body text, then call send_html_email.
# - Dynamic sender sets 'In-Reply-To'/'References' to keep Gmail/Outlook threading.
# - For production, set COMPANY_EMAIL/REPLY_TO to your real inbox so replies reach you.


# =========================
# PART 1 — Imports & Sender Tools (Resend)
# =========================

from dotenv import load_dotenv
from typing import Dict, Optional
from agents import Agent, Runner, trace, function_tool
import os, re, html, asyncio
import resend

# Load .env (API keys, etc.)
load_dotenv(override=True)

# ---- Company context ----
COMPANY_NAME = "All American School of English Teaching"
COMPANY_EMAIL = "onboarding@resend.dev"
TIKTOK_HANDLE = "@all.american.eng"
INSTAGRAM_HANDLE = "all.american.eng"
TIKTOK_URL = "https://www.tiktok.com/@all.american.eng"
INSTAGRAM_URL = "https://www.instagram.com/all.american.eng"

PRICE_PLN = int(os.getenv("PRICE_PLN", "120"))
PRICE_USD = int(os.getenv("PRICE_USD", "25"))

OFFERS = ["General English", "Business English", "Technology English", "Matura prep", "TOEFL"]


os.environ.setdefault("RESEND_SUBJECT", "Want to speak native-level American English?")
os.environ.setdefault("REPLY_TO", COMPANY_EMAIL)  # where replies should go / not complete 

# ---------- Utilities ----------
def _html_to_text_fallback(html_body: str) -> str:
    """Light HTML→text fallback to improve deliverability."""
    txt = html.unescape(html_body)
    txt = re.sub(r'(?i)<br\s*/?>', '\n', txt)
    txt = re.sub(r'(?i)</p\s*>', '\n\n', txt)
    txt = re.sub(
        r'(?is)<a\s+[^>]*href=["\']([^"\']+)["\'][^>]*>(.*?)</a>',
        lambda m: f"{re.sub('<[^>]+>', '', m.group(2)).strip()} ({m.group(1)})",
        txt
    )
    txt = re.sub(r'<[^>]+>', '', txt)
    txt = re.sub(r'\n{3,}', '\n\n', txt).strip()
    return txt


import html as _html
import re as _re

def _paragraphize(text: str) -> str:
    """Turn plain text (even if it contains <br> or </p>) into <p> blocks."""
    t = (text or "").strip()
    t = t.replace("\r\n", "\n").replace("\r", "\n")
    t = _re.sub(r"(?i)<br\s*/?>", "\n", t)
    t = _re.sub(r"(?i)</p\s*>", "\n\n", t)
    t = _re.sub(r"(?i)</?p[^>]*>", "", t)
    t = _re.sub(r"<[^>]+>", "", t)

    # Escape and wrap into paragraphs
    safe = _html.escape(t)
    paras = [p.strip().replace("\n", "<br>") for p in _re.split(r"\n\s*\n+", safe) if p.strip()]
    if not paras:
        return "<p></p>"
    return "\n".join(
        f"<p style='margin:0 0 12px 0; line-height:1.55; font-size:16px; color:#111;'>{p}</p>"
        for p in paras
    )


def _build_html_email(body_text: str) -> str:
    """Minimal, mobile-friendly HTML email with branding, CTA, offer block, and socials."""
    body_html = _paragraphize(body_text)
    return f"""<!doctype html>
<html>
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width,initial-scale=1">
  <title>{_html.escape(COMPANY_NAME)}</title>
</head>
<body style="margin:0;padding:0;background:#f6f7fb;font-family:-apple-system,BlinkMacSystemFont,'Segoe UI',Roboto,Helvetica,Arial,sans-serif;">
  <table role="presentation" width="100%" cellspacing="0" cellpadding="0" border="0">
    <tr>
      <td align="center" style="padding:24px 12px;">
        <table role="presentation" width="600" cellspacing="0" cellpadding="0" border="0" style="max-width:100%;background:#fff;border-radius:12px;overflow:hidden;box-shadow:0 4px 18px rgba(0,0,0,0.06);">
          <tr>
            <td style="background:#0ea5e9;color:#fff;padding:14px 20px;font-weight:600;font-size:16px;">
              {COMPANY_NAME}
            </td>
          </tr>
          <tr>
            <td style="padding:20px 24px;">
              {body_html}
            </td>
          </tr>
          <tr>
            <td style="padding:0 24px 16px 24px;">
              <a href="mailto:{COMPANY_EMAIL}" target="_blank"
                 style="display:inline-block;background:#0ea5e9;color:#fff;text-decoration:none;padding:12px 18px;border-radius:10px;font-weight:600;">
                 Book a lesson
              </a>
            </td>
          </tr>
          <tr>
            <td style="padding:0 24px 20px 24px;font-size:14px;color:#444;line-height:1.55;">
              <strong>What we offer</strong>
              <ul style="margin:8px 0 0 18px;padding:0;">
                {"".join(f"<li>{_html.escape(o)}</li>" for o in OFFERS)}
              </ul>
              <p style="margin:8px 0 0 0;"><strong>Price:</strong> {PRICE_USD} USD per hour</p>
            </td>
          </tr>
          <tr>
            <td style="padding:0 24px 20px 24px;font-size:13px;color:#555;">
              Contact: <a href="mailto:{COMPANY_EMAIL}">{COMPANY_EMAIL}</a><br>
              TikTok: <a href="{TIKTOK_URL}">{TIKTOK_HANDLE}</a> &nbsp;•&nbsp;
              Instagram: <a href="{INSTAGRAM_URL}">{INSTAGRAM_HANDLE}</a>
            </td>
          </tr>
        </table>
        <div style="max-width:600px;width:100%;font-size:12px;color:#6b7280;margin-top:12px;line-height:1.4;">
          You received this because you might be interested in English lessons. If not, please ignore.
        </div>
      </td>
    </tr>
  </table>
</body>
</html>"""

@function_tool
def subject_writer(message: str) -> str:
    """Fast subject line: use env default or fallback."""
    return os.environ.get("RESEND_SUBJECT", "Want to speak native-level American English?")

@function_tool
def html_converter(body: str) -> str:
    """Fast HTML converter using the fixed template (no LLM call)."""
    return _build_html_email(body)

# ---------- Standard sender used by Email Manager ----------
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """
    Send HTML email via Resend to the default recipient (RESEND_TO).
    Includes reply_to so responses go to your inbox.
    """
    api_key = os.environ.get("RESEND_API_KEY")
    if not api_key:
        return {"status": "failure", "message": "RESEND_API_KEY not set in environment."}
    resend.api_key = api_key

    from_email = os.environ.get("RESEND_FROM", COMPANY_EMAIL)  # must be verified in Resend
    to_email   = os.environ.get("RESEND_TO", "miketayloriii15@gmail.com")
    subject    = (subject or "").strip() or os.environ.get("RESEND_SUBJECT", "English lessons")

    text_body = _html_to_text_fallback(html_body)

    try:
        resp = resend.Emails.send({
            "from": from_email,
            "to": [to_email],
            "subject": subject,
            "text": text_body,
            "html": html_body,
            "reply_to": os.environ.get("REPLY_TO", COMPANY_EMAIL),
        })
        return {"status": "success", "id": (resp or {}).get("id"), "to": to_email, "from": from_email, "subject": subject}
    except Exception as e:
        r = getattr(e, "response", None)
        return {"status": "failure", "message": repr(e), "status_code": getattr(r, "status_code", None), "response": getattr(r, "text", None)}

# ---------- Incomplete ----------
@function_tool
def send_html_email_dynamic(
    to_email: str,
    subject: str,
    html_body: str,
    in_reply_to: Optional[str] = None,
    references: Optional[str] = None,
) -> Dict[str, str]:
    """Send HTML email via Resend to an arbitrary recipient, preserving thread headers."""
    api_key = os.environ.get("RESEND_API_KEY")
    if not api_key:
        return {"status": "failure", "message": "RESEND_API_KEY not set in environment."}
    resend.api_key = api_key

    from_email = os.environ.get("RESEND_FROM", COMPANY_EMAIL)  
    payload = {
        "from": from_email,
        "to": [to_email],
        "subject": (subject or "").strip() or os.environ.get("RESEND_SUBJECT", "Re: English lessons"),
        "text": _html_to_text_fallback(html_body),
        "html": html_body,
        "reply_to": os.environ.get("REPLY_TO", COMPANY_EMAIL),
    }

    headers = {}
    if in_reply_to:
        headers["In-Reply-To"] = in_reply_to
    if references:
        headers["References"] = references
    if headers:
        payload["headers"] = headers  

    try:
        resp = resend.Emails.send(payload)
        return {"status": "success", "id": (resp or {}).get("id"), "to": to_email}
    except Exception as e:
        r = getattr(e, "response", None)
        return {"status": "failure", "message": repr(e), "status_code": getattr(r, "status_code", None), "response": getattr(r, "text", None)}



In [43]:
# All American School of English — Agents & Email Manager
# -------------------------------------------------------
# What this part defines:
# 1) Three draft-generating SALES AGENTS (plain text only):
#    - sales_agent1: benefit-led
#    - sales_agent2: social-proof
#    - sales_agent3: concise
# 2) Email pipeline tools (from Part 1 — function tools, no extra LLM calls):
#    - subject_writer  -> subject line
#    - html_converter  -> branded HTML
#    - send_html_email -> send via Resend (to RESEND_TO)
# 3) EMAIL MANAGER agent:
#    - Calls subject_writer once, html_converter once, then send_html_email.
#    - After sending, outputs a one-line confirmation and STOPS.
# 4) REPLY WRITER agent:
#    - Short, context-aware replies (plain text, no HTML, never echo user text).
#    - Uses lesson/teacher facts; proposes time slots; for inbound flow in Part 4.
#
# Notes:
# - Do NOT redefine subject_writer/html_converter here (they come from Part 1).
# - Handoffs = [Email Manager] so other agents can pass the winning draft for send.
# - For debugging, you can print tools/handoffs (lines are included but commented out).

# =========================
# PART 2 — Tools & Email Manager
# =========================

# ---- Three sales agents (draft generators) ----
benefits_style = f"""
You are a cold email writer for {COMPANY_NAME}. Write a concise, plain-text draft (no HTML, no subject).
Audience: potential students who wants to learn english online. Offers: {", ".join(OFFERS)} at {PRICE_USD} USD/hour.
Style: benefit-led (outcomes, confidence, results). 90–140 words. Start with a natural greeting ("Dear {{first_name}}" if provided, else "Hello").
Include one clear CTA to reply or book a trial (mailto:{COMPANY_EMAIL}). Avoid buzzwords; be specific.
"""

social_proof_style = f"""
You are a cold email writer for {COMPANY_NAME}. Plain-text, no subject, no HTML.
Emphasize credibility: TESOL certification, successful student outcomes (e.g., Matura/TOEFL improvements), and 1:1 tailored lessons.
Include offers: {", ".join(OFFERS)} at {PRICE_USD} USD/hour. 90–140 words. One CTA to email {COMPANY_EMAIL}.
"""

concise_style = f"""
You are a cold email writer for {COMPANY_NAME}. Keep it ultra concise (60–90 words), plain-text, no subject, no HTML.
Direct value prop + simple CTA ("Reply 'trial' to book a 30-min session" or email {COMPANY_EMAIL}).
Mention a specific offer only if context hints (Business, Technology, Matura, TOEFL).
"""

sales_agent1 = Agent(name="Sales Agent — Benefits", instructions=benefits_style, model="gpt-4o-mini")
sales_agent2 = Agent(name="Sales Agent — Social Proof", instructions=social_proof_style, model="gpt-4o-mini")
sales_agent3 = Agent(name="Sales Agent — Concise", instructions=concise_style, model="gpt-4o-mini")

sales_tools = [
    sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Benefit-led cold email draft"),
    sales_agent2.as_tool(tool_name="sales_agent2", tool_description="Social-proof cold email draft"),
    sales_agent3.as_tool(tool_name="sales_agent3", tool_description="Concise cold email draft"),
]

# ---- Email pipeline tools for the Email Manager  ----
email_tools = [subject_writer, html_converter, send_html_email]

# ---- Email Manager (formats + sends) ----
email_manager_instructions = (
    "You are an email formatter and sender. You receive the body of an email to be sent. "
    "Step 1: Call subject_writer ONCE to write a subject. "
    "Step 2: Call html_converter ONCE to convert the body to a complete HTML email. "
    "Step 3: Call send_html_email with the subject and HTML body. "
    "After Step 3, OUTPUT exactly a one-line confirmation with the send_html_email 'id' and STOP. "
    "Do not call any more tools after sending. Do not ask follow-up questions."
)

emailer_agent = Agent(
    name="Email Manager",
    instructions=email_manager_instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it"
)

handoffs = [emailer_agent]

# Optional debug
# print("Tools:", sales_tools + email_tools)
# print("Handoffs:", handoffs)

# ---- Reply writer (plain text → short, brand-safe reply) ----
reply_writer = Agent(
    name="Email Reply Writer",
    model="gpt-4o-mini",
    instructions=f"""
You are a concise email reply assistant for {COMPANY_NAME}.

Context facts (ground truth):
- Price: {PRICE_USD} USD/hour.
- Lesson length: 60 minutes.
- Format: Google Meet (online).
- Materials & homework: books/worksheets shared via Google Drive; progress tracking provided.
- Offers: {", ".join(OFFERS)}.
- Teacher: Mike Taylor III — Founder & English Teacher; also Web Developer, Data Scientist, and AI/ML Engineer; from New Orleans, Louisiana (USA); speaks some Polish and German; played professional American football in Europe.

Behavior:
- Read the student's email and answer their specific questions directly and helpfully.
- Keep replies short and skimmable: aim for 60–120 words. Use 2 short paragraphs or brief bullet points if it improves clarity.
- If they ask about how lessons work, include: 60 minutes, on Google Meet, with materials/homework via Google Drive.
- If they ask about the teacher, include ONE short sentence using the teacher facts above (do not list everything unless requested).
- If scheduling is mentioned, propose 2–3 slots in Europe/Warsaw time (e.g., Tue 18:00–18:30, Thu 19:00–19:30, Sat 10:00–10:30) and invite them to choose or suggest alternatives.
- Include a clear CTA to reply with a time or questions, or email {COMPANY_EMAIL}.
- If any crucial detail is missing, ask exactly ONE concise clarifying question.
- Tone: warm, clear, professional. Avoid hype.
- Language: English.
- Never quote or copy the sender’s message; write a fresh answer.
- Output plain text only (no HTML, no markdown).
"""
)





In [44]:
# All American School of English — Turbo Send (fast path)
# -------------------------------------------------------
# What this part does:
# 1) _to_text(res): normalizes Runner.run outputs to plain text (works with many result shapes).
# 2) turbo_send(prompt):
#    - Drafts the email ONCE with sales_agent3 (concise style).
#    - Hands the draft to Email Manager, which calls subject_writer → html_converter → send_html_email.
#    - Prints the final result JSON/confirmation.
#
# Usage:
#   await turbo_send("...your prompt...")   # Notebook/IPython
#   # For scripts: wrap in asyncio.run(turbo_send(...))
#
# Notes:
# - Depends on agents/tools defined in Parts 1–2 (emailer_agent, sales_agent3, subject/html/send tools).
# - Keep max_turns small for speed; increase slightly if you see premature stops.



# =========================
# PART 3 — Turbo pipeline 
# =========================

# Helper: robustly extract text from a Runner.run result
def _to_text(res) -> str:
    if isinstance(res, str):
        return res.strip()

    
    for attr in ("text", "output", "final_output", "content", "message"):
        if hasattr(res, attr):
            val = getattr(res, attr)
            if isinstance(val, str) and val.strip():
                return val.strip()
        
        if hasattr(res, attr) and isinstance(getattr(res, attr), dict):
            d = getattr(res, attr)
            for k in ("text", "content", "message", "result"):
                v = d.get(k)
                if isinstance(v, str) and v.strip():
                    return v.strip()

    
    if isinstance(res, dict):
        for k in ("text", "output", "final_output", "content", "message", "result"):
            v = res.get(k)
            if isinstance(v, str) and v.strip():
                return v.strip()

    
    return str(res).strip()


sales_agents = [sales_agent1, sales_agent2, sales_agent3]

async def turbo_send(prompt: str):
    """One LLM draft -> Email Manager runs subject+HTML+send."""
    with trace("Turbo SDR"):
       
        draft_res = await Runner.run(sales_agent3, prompt, max_turns=1)
        draft_text = _to_text(draft_res)

        
        result = await Runner.run(emailer_agent, draft_text, max_turns=8)

    print("---- TURBO RESULT ----")
    print(result)

# Notebook-safe call:
await turbo_send(
    "Write a cold email for a potential student who wants to have English lessons online; "
    "greet with 'Greetings' and sign from Mike Taylor III — Founder & English Teacher."
)



---- TURBO RESULT ----
RunResult:
- Last agent: Agent(name="Email Manager", ...)
- Final output (str):
    Email sent successfully with ID: a4b93c70-7be5-4bce-baf1-72d013b7155f.
- 7 new item(s)
- 3 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
